# Preproceesing
We preprocess models in this file through the following procedure:
1. Load the model through Open3D
2. Normalize the model such that it fits inside a 32x32x32 voxel grid:
    - Normalize the bounding box to $ [-1.0]^3 $ to $ [1.0]^3 $
    - Voxelize using Open3D
3. Convert the model to an occupancy map and store to a file in human-readable format.

In [56]:
import torch
import numpy as np
import open3d as o3d
import os

o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Warning)

In [57]:
# Read mesh
mesh = o3d.io.read_triangle_mesh("../data/shapenet/1021a0914a7207aff927ed529ad90a11/models/model_normalized.obj")
mesh

TriangleMesh with 7526 points and 8448 triangles.

In [58]:
def normalize(mesh):
    '''
    Normalize mesh in-place.
    '''
    center = mesh.get_center()
    mesh.translate(-center)
    extent = mesh.get_axis_aligned_bounding_box().get_max_extent()
    mesh.scale(2.0 / extent, [0, 0, 0])


AABB = mesh.get_axis_aligned_bounding_box()
print(f"Origin {AABB},\n\t max extent={AABB.get_max_extent()}")

normalize(mesh)

AABB = mesh.get_axis_aligned_bounding_box()
print(f"New {AABB},\n\t max extent={AABB.get_max_extent()}")

Origin AxisAlignedBoundingBox: min: (-0.359457, -0.065165, -0.299895), max: (0.360894, 0.144524, 0.36126),
	 max extent=0.720350980758667
New AxisAlignedBoundingBox: min: (-1.00001, -0.177738, -0.848785), max: (0.999988, 0.404448, 0.986862),
	 max extent=2.0


In [59]:
o3d.visualization.draw_geometries([mesh])

The following part is related to voxelization.

In [68]:
voxel_grid = o3d.geometry.VoxelGrid.create_from_triangle_mesh(mesh, voxel_size=float(2/32 + 0.001))
o3d.visualization.draw_geometries([voxel_grid])

In [61]:
# Fill the voxel_grid
# TODO: Depth Carving

In [87]:
# Convert voxel_grid to numpy matrix
res = np.zeros((32, 32, 32), dtype=np.int32)
for voxel in voxel_grid.get_voxels():
    # print(voxel.grid_index)
    res[tuple(voxel.grid_index)] = 1

print(voxel_grid)
print(np.sum(np.count_nonzero(res)))

VoxelGrid with 433 voxels.
433


In [101]:
np.savetxt('test.txt', res.reshape((1, -1)), fmt='%d', delimiter=' ')

In [92]:
v = np.loadtxt('test.txt', dtype=int, delimiter=' ').reshape((32, 32, 32))
np.sum(np.count_nonzero(v))

433